## 1. 관광지 콘텐츠 기반 추천

In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df=pd.read_csv('천안시 관광지.csv')
df

,순위,광역시/도,시/군/구,관광지명,도로명주소,중분류 카테고리,소분류 카테고리,검색건수
0,1,충청남도,천안시 동남구,천안종합버스터미널,충남 천안시 동남구 만남로 43-0,기타관광,교통시설,243836
1,2,충청남도,천안시 동남구,신세계백화점천안아산점,충남 천안시 동남구 만남로 43-0,쇼핑,백화점,234249
2,3,충청남도,천안시 동남구,독립기념관,충남 천안시 동남구 독립기념관로 1-0,문화관광,전시시설,167873
3,4,충청남도,천안시 서북구,갤러리아백화점센터시티점,충남 천안시 서북구 공원로 227-0,쇼핑,백화점,129023
4,5,충청남도,천안시 서북구,모다아울렛천안아산점,충남 천안시 서북구 공원로 196-0,쇼핑,쇼핑몰,84905
...,...,...,...,...,...,...,...,...
769,770,충청남도,천안시 동남구,대풍농장,충남 천안시 동남구 대흥1길 218-25,체험관광,농/산/어촌체험,3
770,771,충청남도,천안시 동남구,재두농장,충남 천안시 동남구 봉항로 1023-37,체험관광,농/산/어촌체험,3
771,772,충청남도,천안시 동남구,운암사,충남 천안시 동남구 대평1길 113-35,역사관광,종교성지,3
772,773,충청남도,천안시 동남구,신방탁구교실,충남 천안시 동남구 서부대로 257-13,레저스포츠,기타레저스포츠,3


In [11]:
df['내용'] = df['관광지명'] + ' ' + df['중분류 카테고리'] + ' ' + df['소분류 카테고리']

# count 벡터화
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(df['내용'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# 유사도 3위까지
def get_recommendations(idx, sim_matrix):
    sim_scores = list(enumerate(sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:4]  # 자기 자신을 제외하고 상위 3개
    return [i[0] for i in sim_scores]

df['유사 관광지'] = df.index.map(lambda x: get_recommendations(x, cosine_sim))

df['유사 관광지명'] = df['유사 관광지'].apply(lambda x: [df.loc[i, '관광지명'] for i in x])

df

,순위,광역시/도,시/군/구,관광지명,도로명주소,중분류 카테고리,소분류 카테고리,검색건수,내용,유사 관광지,유사 관광지명
0,1,충청남도,천안시 동남구,천안종합버스터미널,충남 천안시 동남구 만남로 43-0,기타관광,교통시설,243836,천안종합버스터미널 기타관광 교통시설,"[16, 33, 432]","[천안고속버스터미널, 성환시외버스터미널, 병천버스터미널]"
1,2,충청남도,천안시 동남구,신세계백화점천안아산점,충남 천안시 동남구 만남로 43-0,쇼핑,백화점,234249,신세계백화점천안아산점 쇼핑 백화점,"[3, 4, 5]","[갤러리아백화점센터시티점, 모다아울렛천안아산점, 천안중앙시장]"
2,3,충청남도,천안시 동남구,독립기념관,충남 천안시 동남구 독립기념관로 1-0,문화관광,전시시설,167873,독립기념관 문화관광 전시시설,"[37, 44, 84]","[천안홍대용과학관, 유관순열사기념관, 리각미술관]"
3,4,충청남도,천안시 서북구,갤러리아백화점센터시티점,충남 천안시 서북구 공원로 227-0,쇼핑,백화점,129023,갤러리아백화점센터시티점 쇼핑 백화점,"[1, 4, 5]","[신세계백화점천안아산점, 모다아울렛천안아산점, 천안중앙시장]"
4,5,충청남도,천안시 서북구,모다아울렛천안아산점,충남 천안시 서북구 공원로 196-0,쇼핑,쇼핑몰,84905,모다아울렛천안아산점 쇼핑 쇼핑몰,"[145, 373, 389]","[폭스캐럿21, 신세계팩토리스토어천안아산점, 동방생활산업]"
...,...,...,...,...,...,...,...,...,...,...,...
769,770,충청남도,천안시 동남구,대풍농장,충남 천안시 동남구 대흥1길 218-25,체험관광,농/산/어촌체험,3,대풍농장 체험관광 농/산/어촌체험,"[32, 78, 134]","[아름다운정원화수목, 풍세관광농원, 산사람농원]"
770,771,충청남도,천안시 동남구,재두농장,충남 천안시 동남구 봉항로 1023-37,체험관광,농/산/어촌체험,3,재두농장 체험관광 농/산/어촌체험,"[32, 78, 134]","[아름다운정원화수목, 풍세관광농원, 산사람농원]"
771,772,충청남도,천안시 동남구,운암사,충남 천안시 동남구 대평1길 113-35,역사관광,종교성지,3,운암사 역사관광 종교성지,"[10, 26, 64]","[각원사, 광덕사, 성불사]"
772,773,충청남도,천안시 동남구,신방탁구교실,충남 천안시 동남구 서부대로 257-13,레저스포츠,기타레저스포츠,3,신방탁구교실 레저스포츠 기타레저스포츠,"[58, 74, 77]","[으뜸풋살신구장, 위너풋살장, 매치업풋살필드]"


In [12]:
df[['유사도1', '유사도2', '유사도3']] = pd.DataFrame(df['유사 관광지명'].tolist(), index=df.index)

df

,순위,광역시/도,시/군/구,관광지명,도로명주소,중분류 카테고리,소분류 카테고리,검색건수,내용,유사 관광지,유사 관광지명,유사도1,유사도2,유사도3
0,1,충청남도,천안시 동남구,천안종합버스터미널,충남 천안시 동남구 만남로 43-0,기타관광,교통시설,243836,천안종합버스터미널 기타관광 교통시설,"[16, 33, 432]","[천안고속버스터미널, 성환시외버스터미널, 병천버스터미널]",천안고속버스터미널,성환시외버스터미널,병천버스터미널
1,2,충청남도,천안시 동남구,신세계백화점천안아산점,충남 천안시 동남구 만남로 43-0,쇼핑,백화점,234249,신세계백화점천안아산점 쇼핑 백화점,"[3, 4, 5]","[갤러리아백화점센터시티점, 모다아울렛천안아산점, 천안중앙시장]",갤러리아백화점센터시티점,모다아울렛천안아산점,천안중앙시장
2,3,충청남도,천안시 동남구,독립기념관,충남 천안시 동남구 독립기념관로 1-0,문화관광,전시시설,167873,독립기념관 문화관광 전시시설,"[37, 44, 84]","[천안홍대용과학관, 유관순열사기념관, 리각미술관]",천안홍대용과학관,유관순열사기념관,리각미술관
3,4,충청남도,천안시 서북구,갤러리아백화점센터시티점,충남 천안시 서북구 공원로 227-0,쇼핑,백화점,129023,갤러리아백화점센터시티점 쇼핑 백화점,"[1, 4, 5]","[신세계백화점천안아산점, 모다아울렛천안아산점, 천안중앙시장]",신세계백화점천안아산점,모다아울렛천안아산점,천안중앙시장
4,5,충청남도,천안시 서북구,모다아울렛천안아산점,충남 천안시 서북구 공원로 196-0,쇼핑,쇼핑몰,84905,모다아울렛천안아산점 쇼핑 쇼핑몰,"[145, 373, 389]","[폭스캐럿21, 신세계팩토리스토어천안아산점, 동방생활산업]",폭스캐럿21,신세계팩토리스토어천안아산점,동방생활산업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,770,충청남도,천안시 동남구,대풍농장,충남 천안시 동남구 대흥1길 218-25,체험관광,농/산/어촌체험,3,대풍농장 체험관광 농/산/어촌체험,"[32, 78, 134]","[아름다운정원화수목, 풍세관광농원, 산사람농원]",아름다운정원화수목,풍세관광농원,산사람농원
770,771,충청남도,천안시 동남구,재두농장,충남 천안시 동남구 봉항로 1023-37,체험관광,농/산/어촌체험,3,재두농장 체험관광 농/산/어촌체험,"[32, 78, 134]","[아름다운정원화수목, 풍세관광농원, 산사람농원]",아름다운정원화수목,풍세관광농원,산사람농원
771,772,충청남도,천안시 동남구,운암사,충남 천안시 동남구 대평1길 113-35,역사관광,종교성지,3,운암사 역사관광 종교성지,"[10, 26, 64]","[각원사, 광덕사, 성불사]",각원사,광덕사,성불사
772,773,충청남도,천안시 동남구,신방탁구교실,충남 천안시 동남구 서부대로 257-13,레저스포츠,기타레저스포츠,3,신방탁구교실 레저스포츠 기타레저스포츠,"[58, 74, 77]","[으뜸풋살신구장, 위너풋살장, 매치업풋살필드]",으뜸풋살신구장,위너풋살장,매치업풋살필드


In [13]:
df.to_excel('천안시 관광지_콘텐츠 추천.xlsx')

## 2. 식당 콘텐츠 기반 추천

In [14]:
df1=pd.read_csv('천안시 음식.csv')
df1

,순위,광역시/도,시/군/구,관광지명,도로명주소,중분류 카테고리,소분류 카테고리,검색건수
0,1,충청남도,천안시 동남구,뚜쥬루빵돌가마점,충남 천안시 동남구 풍세로 706-0,음식,간이음식,81335
1,2,충청남도,천안시 동남구,병천순대거리,충남 천안시 동남구 아우내순대길 33-0,음식,음식점기타,64148
2,3,충청남도,천안시 동남구,박순자아우내순대,충남 천안시 동남구 아우내순대길 47-0,음식,한식,27167
3,4,충청남도,천안시 동남구,랜드마크195,충남 천안시 동남구 유량로 195-0,음식,카페/찻집,24727
4,5,충청남도,천안시 서북구,뚜쥬루성정점,충남 천안시 서북구 백석로 270-0,음식,간이음식,23506
...,...,...,...,...,...,...,...,...
996,997,충청남도,천안시 동남구,큰집추어탕,충남 천안시 동남구 서부대로 181-0,음식,한식,520
997,998,충청남도,천안시 동남구,김정열전통갈비탕,충남 천안시 동남구 청수10길 4-60,음식,한식,518
998,999,충청남도,천안시 동남구,해족왕신방점,충남 천안시 동남구 수곡로 67-0,음식,전문음식,514
999,1000,충청남도,천안시 동남구,파리바게뜨천안병천점,충남 천안시 동남구 아우내장터3길 2-0,음식,간이음식,514


In [15]:
df1['내용'] = df1['관광지명'] + ' ' + df1['소분류 카테고리']

# count 벡터화
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(df1['내용'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# 유사도 3위까지
def get_recommendations(idx, sim_matrix):
    sim_scores = list(enumerate(sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:4]  # 자기 자신을 제외하고 상위 3개
    return [i[0] for i in sim_scores]

df1['유사 관광지'] = df1.index.map(lambda x: get_recommendations(x, cosine_sim))

df1['유사 관광지명'] = df1['유사 관광지'].apply(lambda x: [df1.loc[i, '관광지명'] for i in x])

df1

,순위,광역시/도,시/군/구,관광지명,도로명주소,중분류 카테고리,소분류 카테고리,검색건수,내용,유사 관광지,유사 관광지명
0,1,충청남도,천안시 동남구,뚜쥬루빵돌가마점,충남 천안시 동남구 풍세로 706-0,음식,간이음식,81335,뚜쥬루빵돌가마점 간이음식,"[4, 6, 9]","[뚜쥬루성정점, 맥도날드천안원성DT점, 맥도날드천안두정역DT점]"
1,2,충청남도,천안시 동남구,병천순대거리,충남 천안시 동남구 아우내순대길 33-0,음식,음식점기타,64148,병천순대거리 음식점기타,"[5, 238, 321]","[두정동먹자골목, 신방동먹자골목, 불당동먹자골목]"
2,3,충청남도,천안시 동남구,박순자아우내순대,충남 천안시 동남구 아우내순대길 47-0,음식,한식,27167,박순자아우내순대 한식,"[710, 42, 51]","[박순자아우내순대, 밥상차려주는집[한식], 알토란한정식[한식]]"
3,4,충청남도,천안시 동남구,랜드마크195,충남 천안시 동남구 유량로 195-0,음식,카페/찻집,24727,랜드마크195 카페/찻집,"[763, 16, 18]","[숨, 스타벅스천안불당DT점, 핀스커피]"
4,5,충청남도,천안시 서북구,뚜쥬루성정점,충남 천안시 서북구 백석로 270-0,음식,간이음식,23506,뚜쥬루성정점 간이음식,"[0, 6, 9]","[뚜쥬루빵돌가마점, 맥도날드천안원성DT점, 맥도날드천안두정역DT점]"
...,...,...,...,...,...,...,...,...,...,...,...
996,997,충청남도,천안시 동남구,큰집추어탕,충남 천안시 동남구 서부대로 181-0,음식,한식,520,큰집추어탕 한식,"[42, 51, 79]","[밥상차려주는집[한식], 알토란한정식[한식], 숟가락반상마실천안본점 [한식]]"
997,998,충청남도,천안시 동남구,김정열전통갈비탕,충남 천안시 동남구 청수10길 4-60,음식,한식,518,김정열전통갈비탕 한식,"[42, 51, 79]","[밥상차려주는집[한식], 알토란한정식[한식], 숟가락반상마실천안본점 [한식]]"
998,999,충청남도,천안시 동남구,해족왕신방점,충남 천안시 동남구 수곡로 67-0,음식,전문음식,514,해족왕신방점 전문음식,"[381, 8, 17]","[통, 충남집순대, 쿠우쿠우천안불당점]"
999,1000,충청남도,천안시 동남구,파리바게뜨천안병천점,충남 천안시 동남구 아우내장터3길 2-0,음식,간이음식,514,파리바게뜨천안병천점 간이음식,"[0, 4, 6]","[뚜쥬루빵돌가마점, 뚜쥬루성정점, 맥도날드천안원성DT점]"


In [16]:
df1[['유사도1', '유사도2', '유사도3']] = pd.DataFrame(df1['유사 관광지명'].tolist(), index=df1.index)

df1

,순위,광역시/도,시/군/구,관광지명,도로명주소,중분류 카테고리,소분류 카테고리,검색건수,내용,유사 관광지,유사 관광지명,유사도1,유사도2,유사도3
0,1,충청남도,천안시 동남구,뚜쥬루빵돌가마점,충남 천안시 동남구 풍세로 706-0,음식,간이음식,81335,뚜쥬루빵돌가마점 간이음식,"[4, 6, 9]","[뚜쥬루성정점, 맥도날드천안원성DT점, 맥도날드천안두정역DT점]",뚜쥬루성정점,맥도날드천안원성DT점,맥도날드천안두정역DT점
1,2,충청남도,천안시 동남구,병천순대거리,충남 천안시 동남구 아우내순대길 33-0,음식,음식점기타,64148,병천순대거리 음식점기타,"[5, 238, 321]","[두정동먹자골목, 신방동먹자골목, 불당동먹자골목]",두정동먹자골목,신방동먹자골목,불당동먹자골목
2,3,충청남도,천안시 동남구,박순자아우내순대,충남 천안시 동남구 아우내순대길 47-0,음식,한식,27167,박순자아우내순대 한식,"[710, 42, 51]","[박순자아우내순대, 밥상차려주는집[한식], 알토란한정식[한식]]",박순자아우내순대,밥상차려주는집[한식],알토란한정식[한식]
3,4,충청남도,천안시 동남구,랜드마크195,충남 천안시 동남구 유량로 195-0,음식,카페/찻집,24727,랜드마크195 카페/찻집,"[763, 16, 18]","[숨, 스타벅스천안불당DT점, 핀스커피]",숨,스타벅스천안불당DT점,핀스커피
4,5,충청남도,천안시 서북구,뚜쥬루성정점,충남 천안시 서북구 백석로 270-0,음식,간이음식,23506,뚜쥬루성정점 간이음식,"[0, 6, 9]","[뚜쥬루빵돌가마점, 맥도날드천안원성DT점, 맥도날드천안두정역DT점]",뚜쥬루빵돌가마점,맥도날드천안원성DT점,맥도날드천안두정역DT점
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,997,충청남도,천안시 동남구,큰집추어탕,충남 천안시 동남구 서부대로 181-0,음식,한식,520,큰집추어탕 한식,"[42, 51, 79]","[밥상차려주는집[한식], 알토란한정식[한식], 숟가락반상마실천안본점 [한식]]",밥상차려주는집[한식],알토란한정식[한식],숟가락반상마실천안본점 [한식]
997,998,충청남도,천안시 동남구,김정열전통갈비탕,충남 천안시 동남구 청수10길 4-60,음식,한식,518,김정열전통갈비탕 한식,"[42, 51, 79]","[밥상차려주는집[한식], 알토란한정식[한식], 숟가락반상마실천안본점 [한식]]",밥상차려주는집[한식],알토란한정식[한식],숟가락반상마실천안본점 [한식]
998,999,충청남도,천안시 동남구,해족왕신방점,충남 천안시 동남구 수곡로 67-0,음식,전문음식,514,해족왕신방점 전문음식,"[381, 8, 17]","[통, 충남집순대, 쿠우쿠우천안불당점]",통,충남집순대,쿠우쿠우천안불당점
999,1000,충청남도,천안시 동남구,파리바게뜨천안병천점,충남 천안시 동남구 아우내장터3길 2-0,음식,간이음식,514,파리바게뜨천안병천점 간이음식,"[0, 4, 6]","[뚜쥬루빵돌가마점, 뚜쥬루성정점, 맥도날드천안원성DT점]",뚜쥬루빵돌가마점,뚜쥬루성정점,맥도날드천안원성DT점


In [17]:
df1.to_excel('천안시 음식_콘텐츠 추천.xlsx')

## 3.숙소 콘텐츠 기반 추천

In [23]:
df2=pd.read_excel('천안시 숙소.xlsx')
df2

,광역시/도,시/군/구,관광지명,도로명주소,Latitude,Longitude,중분류 카테고리,소분류 카테고리,검색건수,반려동물 가능여부
0,충청남도,천안시 동남구,더메인즈호텔,충남 천안시 동남구 청수11로 34-0,36.782387,127.156230,숙박,호텔,7157,1
1,충청남도,천안시 동남구,상록호텔,충남 천안시 동남구 수신로 576-0,36.740781,127.288472,숙박,호텔,4344,0
2,충청남도,천안시 동남구,브라운도트호텔천안터미널점,충남 천안시 동남구 신부3길 25-0,36.818577,127.160030,숙박,호텔,3776,1
3,충청남도,천안시 동남구,웨스턴호텔,충남 천안시 동남구 신부2길 25-0,36.818421,127.158947,숙박,호텔,2987,0
4,충청남도,천안시 동남구,호텔캄캄,충남 천안시 동남구 신부2길 31-0,36.818762,127.159571,숙박,호텔,2022,0
...,...,...,...,...,...,...,...,...,...,...
147,충청남도,천안시 동남구,캠핑하우스,충남 천안시 동남구 용곡1길 98-0,36.792595,127.144479,숙박,캠핑,56,1
148,충청남도,천안시 동남구,천안내뜰캠핑장,충남 천안시 동남구 동산광덕길 312-0,36.794373,127.337391,숙박,캠핑,13,1
149,충청남도,천안시 동남구,광덕산풍경캠핑장,충남 천안시 동남구 해수길 178-47,36.675549,127.027894,숙박,캠핑,3,1
150,충청남도,천안시 서북구,파라프라,충남 천안시 서북구 상덕로 42-7,36.869564,127.120069,숙박,캠핑,3209,1


In [24]:
df2['내용'] = df2['관광지명'] + ' ' + df2['소분류 카테고리'] + ' ' + df2['반려동물 가능여부'].astype(str)

# count 벡터화
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(df2['내용'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# 유사도 상위 3위까지
def get_recommendations(idx, sim_matrix):
    sim_scores = list(enumerate(sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:4]  # 자기 자신을 제외하고 상위 3개
    return [i[0] for i in sim_scores]

df2['유사 관광지'] = df2.index.map(lambda x: get_recommendations(x, cosine_sim))

df2['유사 관광지명'] = df2['유사 관광지'].apply(lambda x: [df2.loc[i, '관광지명'] for i in x])

df2

,광역시/도,시/군/구,관광지명,도로명주소,Latitude,Longitude,중분류 카테고리,소분류 카테고리,검색건수,반려동물 가능여부,내용,유사 관광지,유사 관광지명
0,충청남도,천안시 동남구,더메인즈호텔,충남 천안시 동남구 청수11로 34-0,36.782387,127.156230,숙박,호텔,7157,1,더메인즈호텔 호텔 1,"[1, 2, 3]","[상록호텔, 브라운도트호텔천안터미널점, 웨스턴호텔]"
1,충청남도,천안시 동남구,상록호텔,충남 천안시 동남구 수신로 576-0,36.740781,127.288472,숙박,호텔,4344,0,상록호텔 호텔 0,"[0, 2, 3]","[더메인즈호텔, 브라운도트호텔천안터미널점, 웨스턴호텔]"
2,충청남도,천안시 동남구,브라운도트호텔천안터미널점,충남 천안시 동남구 신부3길 25-0,36.818577,127.160030,숙박,호텔,3776,1,브라운도트호텔천안터미널점 호텔 1,"[0, 1, 3]","[더메인즈호텔, 상록호텔, 웨스턴호텔]"
3,충청남도,천안시 동남구,웨스턴호텔,충남 천안시 동남구 신부2길 25-0,36.818421,127.158947,숙박,호텔,2987,0,웨스턴호텔 호텔 0,"[0, 1, 2]","[더메인즈호텔, 상록호텔, 브라운도트호텔천안터미널점]"
4,충청남도,천안시 동남구,호텔캄캄,충남 천안시 동남구 신부2길 31-0,36.818762,127.159571,숙박,호텔,2022,0,호텔캄캄 호텔 0,"[0, 1, 2]","[더메인즈호텔, 상록호텔, 브라운도트호텔천안터미널점]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,충청남도,천안시 동남구,캠핑하우스,충남 천안시 동남구 용곡1길 98-0,36.792595,127.144479,숙박,캠핑,56,1,캠핑하우스 캠핑 1,"[125, 126, 127]","[독립기념관야영장, 아우내관광농원캠핑장, 천안국민여가캠핑장]"
148,충청남도,천안시 동남구,천안내뜰캠핑장,충남 천안시 동남구 동산광덕길 312-0,36.794373,127.337391,숙박,캠핑,13,1,천안내뜰캠핑장 캠핑 1,"[125, 126, 127]","[독립기념관야영장, 아우내관광농원캠핑장, 천안국민여가캠핑장]"
149,충청남도,천안시 동남구,광덕산풍경캠핑장,충남 천안시 동남구 해수길 178-47,36.675549,127.027894,숙박,캠핑,3,1,광덕산풍경캠핑장 캠핑 1,"[125, 126, 127]","[독립기념관야영장, 아우내관광농원캠핑장, 천안국민여가캠핑장]"
150,충청남도,천안시 서북구,파라프라,충남 천안시 서북구 상덕로 42-7,36.869564,127.120069,숙박,캠핑,3209,1,파라프라 캠핑 1,"[125, 126, 127]","[독립기념관야영장, 아우내관광농원캠핑장, 천안국민여가캠핑장]"


In [25]:
df2[['유사도1', '유사도2', '유사도3']] = pd.DataFrame(df2['유사 관광지명'].tolist(), index=df2.index)

df2

,광역시/도,시/군/구,관광지명,도로명주소,Latitude,Longitude,중분류 카테고리,소분류 카테고리,검색건수,반려동물 가능여부,내용,유사 관광지,유사 관광지명,유사도1,유사도2,유사도3
0,충청남도,천안시 동남구,더메인즈호텔,충남 천안시 동남구 청수11로 34-0,36.782387,127.156230,숙박,호텔,7157,1,더메인즈호텔 호텔 1,"[1, 2, 3]","[상록호텔, 브라운도트호텔천안터미널점, 웨스턴호텔]",상록호텔,브라운도트호텔천안터미널점,웨스턴호텔
1,충청남도,천안시 동남구,상록호텔,충남 천안시 동남구 수신로 576-0,36.740781,127.288472,숙박,호텔,4344,0,상록호텔 호텔 0,"[0, 2, 3]","[더메인즈호텔, 브라운도트호텔천안터미널점, 웨스턴호텔]",더메인즈호텔,브라운도트호텔천안터미널점,웨스턴호텔
2,충청남도,천안시 동남구,브라운도트호텔천안터미널점,충남 천안시 동남구 신부3길 25-0,36.818577,127.160030,숙박,호텔,3776,1,브라운도트호텔천안터미널점 호텔 1,"[0, 1, 3]","[더메인즈호텔, 상록호텔, 웨스턴호텔]",더메인즈호텔,상록호텔,웨스턴호텔
3,충청남도,천안시 동남구,웨스턴호텔,충남 천안시 동남구 신부2길 25-0,36.818421,127.158947,숙박,호텔,2987,0,웨스턴호텔 호텔 0,"[0, 1, 2]","[더메인즈호텔, 상록호텔, 브라운도트호텔천안터미널점]",더메인즈호텔,상록호텔,브라운도트호텔천안터미널점
4,충청남도,천안시 동남구,호텔캄캄,충남 천안시 동남구 신부2길 31-0,36.818762,127.159571,숙박,호텔,2022,0,호텔캄캄 호텔 0,"[0, 1, 2]","[더메인즈호텔, 상록호텔, 브라운도트호텔천안터미널점]",더메인즈호텔,상록호텔,브라운도트호텔천안터미널점
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,충청남도,천안시 동남구,캠핑하우스,충남 천안시 동남구 용곡1길 98-0,36.792595,127.144479,숙박,캠핑,56,1,캠핑하우스 캠핑 1,"[125, 126, 127]","[독립기념관야영장, 아우내관광농원캠핑장, 천안국민여가캠핑장]",독립기념관야영장,아우내관광농원캠핑장,천안국민여가캠핑장
148,충청남도,천안시 동남구,천안내뜰캠핑장,충남 천안시 동남구 동산광덕길 312-0,36.794373,127.337391,숙박,캠핑,13,1,천안내뜰캠핑장 캠핑 1,"[125, 126, 127]","[독립기념관야영장, 아우내관광농원캠핑장, 천안국민여가캠핑장]",독립기념관야영장,아우내관광농원캠핑장,천안국민여가캠핑장
149,충청남도,천안시 동남구,광덕산풍경캠핑장,충남 천안시 동남구 해수길 178-47,36.675549,127.027894,숙박,캠핑,3,1,광덕산풍경캠핑장 캠핑 1,"[125, 126, 127]","[독립기념관야영장, 아우내관광농원캠핑장, 천안국민여가캠핑장]",독립기념관야영장,아우내관광농원캠핑장,천안국민여가캠핑장
150,충청남도,천안시 서북구,파라프라,충남 천안시 서북구 상덕로 42-7,36.869564,127.120069,숙박,캠핑,3209,1,파라프라 캠핑 1,"[125, 126, 127]","[독립기념관야영장, 아우내관광농원캠핑장, 천안국민여가캠핑장]",독립기념관야영장,아우내관광농원캠핑장,천안국민여가캠핑장


In [26]:
df2.to_excel('천안시 숙소_콘텐츠 추천.xlsx')